In [ ]:
using Revise
using QuanticsTCI
using Test
using StaticArrays
using BenchmarkTools

In [ ]:
divrem(2, 3)

In [ ]:
R = 4
d = 3
B = 2

# Grid on [1, 2, ..., d^R]
grid = QuanticsTCI.InherentDiscreteGrid{d,B}(R)

In [ ]:
x = [1, 1, 1, 4]
@benchmark QuanticsTCI.quantics_to_index_fused(Val(B), Val(d), $x)

In [ ]:
d = 1
R = 2 # number of bits
B = 3

index_reconst = Int[]
for index in 1:B^R
    bitlist_ = QuanticsTCI.index_to_quantics(Val(B), index; numdigits=R)
    @show index
    @show bitlist_
    @show @allocated quantics_to_index_fused(Val(B), Val(d), bitlist_)
    push!(index_reconst, only(quantics_to_index_fused(Val(B), Val(d), bitlist_)))
end

collect(1:B^R) == index_reconst


In [ ]:
base = 3
dim = 2
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 1]) == (1, 1)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 2]) == (2, 1)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 3]) == (3, 1)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 4]) == (1, 2)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 5]) == (2, 2)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 6]) == (3, 2)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 7]) == (1, 3)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 8]) == (2, 3)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 9]) == (3, 3)
@show QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [2, 1]) == (4, 1)

In [ ]:
base = 2
dim = 2
R = 2

for j in 1:base^R, i in 1:base^R
    index = (i, j)
    digitlist = Vector{Int}(undef, R)
    QuanticsTCI.index_to_quantics_fused!(Val(base), digitlist, index)
    index_reconst = QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), digitlist)
    @test index == index_reconst
end
#@test QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 1]) == (1, 1)
#@test QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 2]) == (2, 1)
#@test QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 3]) == (3, 1)
#@test QuanticsTCI.quantics_to_index_fused(Val(base), Val(dim), [1, 4]) == (1, 2)

In [ ]:
# index is one-based. The result is also one-based.
# position must be one-based: 1 is the least (most right) significant digit.
function digit_at_index(::Val{base}, index, position)
    return mod((index-1), base^position) ÷ base^(position-1) + 1
end

i = 3
@show digit_at_index(Val(2), i, 1)
@show digit_at_index(Val(2), i, 2)
@show digit_at_index(Val(2), i, 3)
@show digit_at_index(Val(2), i, 4)

In [ ]:
x = ones(Int, R)
@benchmark qf($x)

In [ ]:
#@benchmark QuanticsTCI.quantics_to_origcoord_fused(grid, $x)
@benchmark QuanticsTCI.quantics_to_index_fused(Val(B), Val(d), $x)

In [ ]:
qf2 = QuanticsTCI.quantics_function_fused(Float64, grid, f)

x = ones(Int, R)
@benchmark qf2($x)

In [ ]:
function testf(x, y, z)
    res = zeros(MVector{1, Float64})
    res .= x .+ y .+ z
    return res
end

a = (1,)
b = (1,)
c = (1,)

@benchmark testf($a, $b, $c)

In [ ]:
B = 3
R = 4
for index in 1:B^R
    bitlist = [mod(index, B^(R-i+1)) ÷ B^(R-i) + 1 for i in 1:R]
    @show bitlist
end

In [ ]:
x = ones(Int, 10)
y = ones(Int, 10)
z = ones(Int, 20)
xy = [x, y]
@benchmark QuanticsTCI.deinterleave_dimensions!(xy, z)

In [ ]:
n = 2
np = 2^n
for i1 in [1,2], i2 in 1:2
    q1 = index_to_quantics(i1, n)
    q2 = index_to_quantics(i2, n)
    @test fuse_dimensions(q1, q2) == index_to_quantics_fused([i1, i2], n)
    @test interleave_dimensions(q1, q2) == index_to_quantics_interleaved([i1, i2], n)
    @show i1
    @show q1
    @show i2
    @show q2
    @show fuse_dimensions(q1, q2)
end

In [ ]:
n = 2
i1, i2 = 1, 1
q1 = index_to_quantics(i1, n)
q2 = index_to_quantics(i2, n)
@benchmark fuse_dimensions($q1, $q2)

In [ ]:
result = ones(Int, n)
@benchmark QuanticsTCI.fuse_dimensions!($result, $q1, $q2)

In [ ]:
n = 10
i1, i2 = 1, 1
q1 = index_to_quantics(i1, n)
q2 = index_to_quantics(i2, n)
q12 = fuse_dimensions(q1, q2)

In [ ]:
@benchmark split_dimensions(Val(2), q12, 2)

In [ ]:
@test split_dimensions([1, 1, 1, 1], 1) == [[1, 1, 1, 1]]
@test split_dimensions([1, 1, 1, 1], 3) == [[1, 1, 1, 1] for i in 1:3]
@test split_dimensions([2, 2, 2, 2], 3) == [[2, 2, 2, 2], [1, 1, 1, 1], [1, 1, 1, 1]]
@test split_dimensions([3, 3, 3, 3], 3) == [[1, 1, 1, 1], [2, 2, 2, 2], [1, 1, 1, 1]]
@test split_dimensions([5, 5, 5, 5], 3) == [[1, 1, 1, 1], [1, 1, 1, 1], [2, 2, 2, 2]]
@test split_dimensions([1, 2, 4, 8], 3) == [[1, 2, 2, 2], [1, 1, 2, 2], [1, 1, 1, 2]]

In [ ]:
result = split_dimensions([1, 1, 1, 1], 3)
@show result
bitlist = [1, 1, 1, 1]
@benchmark QuanticsTCI.split_dimensions!(Val(2), result, bitlist)

In [ ]:
result